# Text Classification with Cleanlab, TensorFlow, & SciKeras

This tutorial will use Cleanlab to find potential label errors in the IMDb movie review dataset. This dataset contains 50,000 labeled text reviews split evenly in the train and test set. Each review is labeled with a binary sentiment polarity label - positive (1) or negative (0). Cleanlab will shortlist *hundreds* of examples that confuses our ML model the most; many of which are potential label errors, edge cases and obscure examples.

**Overview of what we'll do in this tutorial:**

- Build a simple TensorFlow & Keras neural net and wrap it with SciKeras to make it scikit-learn compatible.

- Compute the out-of-sample predicted probabilities, ``pyx``, with cross validation.

- Generate a list of potential label errors with Cleanlab's ``get_noise_indices``.

- Build and train aa robust model with Cleanlab's ``LearningWithNoisyLabels`` wrapper. 

**Data:** https://ai.stanford.edu/~amaas/data/sentiment/

In [1]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

## **1. Install the required dependencies**

``%%capture`` is a magic function to hide the cell's output.

In [2]:
%%capture

%pip install cleanlab sklearn pandas tensorflow tensorflow-datasets scikeras

## **2. Load the ACL's IMDb dataset**

Load the full dataset from TensorFlow Dataset. 

In [3]:
%%capture

import tensorflow_datasets as tfds

raw_full_ds = tfds.load(name='imdb_reviews', split=('train+test'), batch_size=-1, as_supervised=True)

Split the dataset into two numpy arrays:
1. ``raw_full_texts`` for the movie reviews in text format, and
2. ``full_labels`` for the labels.

In [4]:
raw_full_texts, full_labels = tfds.as_numpy(raw_full_ds)

## **3. Preprocess the text data**

Define a function that can standardize the text data in three steps:
1. Convert it to lower case
2. Remove the HTML break tags, ``<br />``
3. Remove any punctuation marks

In [5]:
import tensorflow as tf
import re
import string

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html, f'[{re.escape(string.punctuation)}]','')

Create a ``TextVectorization`` layer that can standardize (by running the ``custom_standardization`` function we've just defined above), tokenize and vectorize our text data.

In [6]:
from tensorflow.keras import layers

max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

Adapting ``vectorize_layer`` on our text data creates a mapping of each token to an integer. After that, we can vectorize our text data with the adapted ``vectorize_layer``. Finally, we'll also convert our text data into a numpy array as required by Cleanlab.

In [7]:
vectorize_layer.adapt(raw_full_texts)

full_texts = vectorize_layer(raw_full_texts)

full_texts = full_texts.numpy()

## **4. Build a classifcation model**

Here, we build and compile a simple neural network with TensorFlow and Keras.

In [8]:
from tensorflow.keras import losses, metrics

model = tf.keras.Sequential([
  tf.keras.Input(shape=(None,), dtype="int64"),
  layers.Embedding(max_features + 1, 16),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.compile(optimizer='adam',
              loss=losses.BinaryCrossentropy(from_logits=True),
              metrics=metrics.BinaryAccuracy())

## **5. Wrap with SciKeras for scikit-learn compatibility**

As some of Cleanlab's feature requires scikit-learn compatibility, we will need to adapt the above TensorFlow & Keras neural net accordingly. SciKeras is a convenient package that helps with this, read more about it here: https://www.adriangb.com/scikeras/stable/

In [9]:
from scikeras.wrappers import KerasClassifier

model = KerasClassifier(model, epochs=10)

## **6. Compute the out-of-sample predicted probabilities with cross validation**

We will fit the entire dataset on the model used to compute the out-of-sample predicted probabilities, ``pyx``, with cross validation. This model will not be used for model evaluation.

In [10]:
_ = model.fit(full_texts, full_labels)

Epoch 1/10


   1/1563 [..............................] - ETA: 13:35 - loss: 0.6948 - binary_accuracy: 0.5625

  18/1563 [..............................] - ETA: 4s - loss: 0.6933 - binary_accuracy: 0.5087   

  35/1563 [..............................] - ETA: 4s - loss: 0.6928 - binary_accuracy: 0.5125

  52/1563 [..............................] - ETA: 4s - loss: 0.6924 - binary_accuracy: 0.4976

...

1500/1563 [===========================>..] - ETA: 0s - loss: 0.2038 - binary_accuracy: 0.9185

1519/1563 [============================>.] - ETA: 0s - loss: 0.2038 - binary_accuracy: 0.9185

1538/1563 [============================>.] - ETA: 0s - loss: 0.2039 - binary_accuracy: 0.9183

1557/1563 [============================>.] - ETA: 0s - loss: 0.2045 - binary_accuracy: 0.9180

1563/1563 [==============================] - 4s 3ms/step - loss: 0.2043 - binary_accuracy: 0.9182


Compute the out-of-sample predicted probabilities, ``pyx``, with cross validation.

In [11]:
%%capture

from sklearn.model_selection import cross_val_predict

pyx = cross_val_predict(model, full_texts, full_labels, cv=3, method='predict_proba')

## **7. Run Cleanlab's to find potential label errors**

Cleanlab has a ``get_noise_indices`` function to generate a list of potential label errors. Setting ``sorted_index_method="prob_given_label"`` returns the indices of all the most likely label errors, sorted by the most suspicious example first.

In [12]:
from cleanlab.pruning import get_noise_indices

ordered_label_errors = get_noise_indices(
    s=full_labels,
    psx=pyx,
    sorted_index_method="prob_given_label"
)

## **8. Review some of the highest potential label errors**

In [13]:
print(f"Cleanlab found {len(ordered_label_errors)} potential label errors. Here are the indices of the top 10 most likely ones: \n {ordered_label_errors[:10]}")

Cleanlab found 505 potential label errors. Here are the indices of the top 10 most likely ones: 
 [44582 10404 30151 29182 37856 16633 22370 43777 13853  2468]


Change pandas display max column width to ``None`` and define a new function, ``print_as_df``, that can print any example from the raw dataset with just its index number.

In [14]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

def print_as_df(index):
    return pd.DataFrame({'texts': raw_full_texts[index], 'labels': full_labels[index]}, [index])

Review labeled as positive (1), but should be negative (0). Here are some review snippets:

> - "...incredibly **awful** score..."
>
> - "...**worst** Foley work ever done."
>
> - "...script is **incomprehensible**..."
>
> - "...editing is just **bizarre**."
>
> - "...**atrocious** pan and scan..."
>
> - "...**incoherent mess**..."
>
> - "...**amateur** directing there."

In [15]:
print_as_df(44582)

,texts,labels
44582,"b'This movie is stuffed full of stock Horror movie goodies: chained lunatics, pre-meditated murder, a mad (vaguely lesbian) female scientist with an even madder father who wears a mask because of his horrible disfigurement, poisoning, spooky castles, werewolves (male and female), adultery, slain lovers, Tibetan mystics, the half-man/half-plant victim of some unnamed experiment, grave robbing, mind control, walled up bodies, a car crash on a lonely road, electrocution, knights in armour - the lot, all topped off with an incredibly awful score and some of the worst Foley work ever done.<br /><br />The script is incomprehensible (even by badly dubbed Spanish Horror movie standards) and some of the editing is just bizarre. In one scene where the lead female evil scientist goes to visit our heroine in her bedroom for one of the badly dubbed: ""That is fantastical. I do not understand. Explain to me again how this is..."" exposition scenes that litter this movie, there is a sudden hand held cutaway of the girl\'s thighs as she gets out of bed for no apparent reason at all other than to cover a cut in the bad scientist\'s ""Mwahaha! All your werewolfs belong mine!"" speech. Though why they went to the bother I don\'t know because there are plenty of other jarring jump cuts all over the place - even allowing for the atrocious pan and scan of the print I saw.<br /><br />The Director was, according to one interview with the star, drunk for most of the shoot and the film looks like it. It is an incoherent mess. It\'s made even more incoherent by the inclusion of werewolf rampage footage from a different film The Mark of the Wolf Man (made 4 years earlier, featuring the same actor but playing the part with more aggression and with a different shirt and make up - IS there a word in Spanish for ""Continuity""?) and more padding of another actor in the wolfman get-up ambling about in long shot.<br /><br />The music is incredibly bad varying almost at random from full orchestral creepy house music, to bosannova, to the longest piano and gong duet ever recorded. (Thinking about it, it might not have been a duet. It might have been a solo. The piano part was so simple it could have been picked out with one hand while the player whacked away at the gong with the other.) <br /><br />This is one of the most bewilderedly trance-state inducing bad movies of the year so far for me. Enjoy.<br /><br />Favourite line: ""Ilona! This madness and perversity will turn against you!"" How true.<br /><br />Favourite shot: The lover, discovering his girlfriend slain, dropping the candle in a cartoon-like demonstration of surprise. Rank amateur directing there.'",1


Review labeled as positive (1), but should be negative (0). Here are some review snippets:

> - "...film seems **cheap**."
>
> - "...unbelievably **bad**..."
>
> - "...cinematography is **badly** lit..."
>
> - "...everything looking **grainy** and **ugly**."
>
> - "...sound is so **terrible**..."

In [16]:
print_as_df(10404)

,texts,labels
10404,"b'This low-budget erotic thriller that has some good points, but a lot more bad one. The plot revolves around a female lawyer trying to clear her lover who is accused of murdering his wife. Being a soft-core film, that entails her going undercover at a strip club and having sex with possible suspects. As plots go for this type of genre, not to bad. The script is okay, and the story makes enough sense for someone up at 2 AM watching this not to notice too many plot holes. But everything else in the film seems cheap. The lead actors aren\'t that bad, but pretty much all the supporting ones are unbelievably bad (one girl seems like she is drunk and/or high). The cinematography is badly lit, with everything looking grainy and ugly. The sound is so terrible that you can barely hear what people are saying. The worst thing in this movie is the reason you\'re watching it-the sex. The reason people watch these things is for hot sex scenes featuring really hot girls in Red Shoe Diary situations. The sex scenes aren\'t hot they\'re sleazy, shot in that porno style where everything is just a master shot of two people going at it. The woman also look like they are refuges from a porn shoot. I\'m not trying to be rude or mean here, but they all have that breast implants and a burned out/weathered look. Even the title, ""Deviant Obsession"", sounds like a Hardcore flick. Not that I don\'t have anything against porn - in fact I love it. But I want my soft-core and my hard-core separate. What ever happened to actresses like Shannon Tweed, Jacqueline Lovell, Shannon Whirry and Kim Dawson? Women that could act and who would totally arouse you? And what happened to B erotic thrillers like Body Chemistry, Nighteyes and even Stripped to Kill. Sure, none of these where masterpieces, but at least they felt like movies. Plus, they were pushing the envelope, going beyond Hollywood\'s relatively prude stance on sex, sexual obsessions and perversions. Now they just make hard-core films without the hard-core sex.'",1


Review labeled as positive (1), but should be negative (0). Here are some review snippets:

> - "...hard to imagine a **boring** shark movie..."
>
> - "**Poor focus** in some scenes made the production seems **amateurish**."
>
> - "...**do nothing** to take advantage of..."
>
> - "...**far too few** scenes of any depth or variety."
>
> - "...just **look flat**...no contrast of depth..."
>
> - "...**introspective** and **dull**...constant **disappointment**."

In [17]:
print_as_df(30151)

,texts,labels
30151,"b'Like the gentle giants that make up the latter half of this film\'s title, Michael Oblowitz\'s latest production has grace, but it\'s also slow and ponderous. The producer\'s last outing, ""Mosquitoman-3D"" had the same problem. It\'s hard to imagine a boring shark movie, but they somehow managed it. The only draw for Hammerhead: Shark Frenzy was it\'s passable animatronix, which is always fun when dealing with wondrous worlds beneath the ocean\'s surface. But even that was only passable. Poor focus in some scenes made the production seems amateurish. With Dolphins and Whales, the technology is all but wasted. Cloudy scenes and too many close-ups of the film\'s giant subjects do nothing to take advantage of IMAX\'s stunning 3D capabilities. There are far too few scenes of any depth or variety. Close-ups of these awesome creatures just look flat and there is often only one creature in the cameras field, so there is no contrast of depth. Michael Oblowitz is trying to follow in his father\'s footsteps, but when you\'ve got Shark-Week on cable, his introspective and dull treatment of his subjects is a constant disappointment.'",1


Cleanlab has shortlisted the most likely label errors to speed up your data cleaning process. With this list, you can decide whether to fix label errors, augment edge cases or remove obscure examples. 

These human-in-the-loop processes may be time-consuming, so if you'd like Cleanlab to automatically remove these noisy examples and train a model directly on the partially mislabeled dataset, you're in luck! Cleanlab provides a ``LearningWithNoisyLabels`` wrapper to do precisely this:

## **9. Adapt with Cleanlab's wrapper and train a robust model**

Similar to before, we will load the dataset, but this time, we will load the train and test set separately.

In [18]:
raw_train_ds = tfds.load(name='imdb_reviews', split='train', batch_size=-1, as_supervised=True)
raw_test_ds = tfds.load(name='imdb_reviews', split='test', batch_size=-1, as_supervised=True)

raw_train_texts, train_labels = tfds.as_numpy(raw_train_ds)
raw_test_texts, test_labels = tfds.as_numpy(raw_test_ds)

We will use the same ``vectorize_layer`` as before, but we will reset its state and adapt it only on the train set. 

In [19]:
vectorize_layer.reset_state() 

vectorize_layer.adapt(raw_train_texts) 

Vectorize the text data in the train and test sets, then convert them into numpy arrays.

In [20]:
train_texts = vectorize_layer(raw_train_texts) 
test_texts = vectorize_layer(raw_test_texts)

train_texts = train_texts.numpy()
test_texts = test_texts.numpy()

We use the ``clone`` function to construct a new unfitted model then wrap it with Cleanlab's ``LearningWithNoisyLabels`` wrapper.

In [21]:
from sklearn.base import clone
from cleanlab.classification import LearningWithNoisyLabels

model = clone(model)

lnl = LearningWithNoisyLabels(clf=model)

Train the wrapped model, ``lnl``, on the train set. 

In [22]:
_ = lnl.fit(train_texts, train_labels)

Epoch 1/10


  1/625 [..............................] - ETA: 2:49 - loss: 0.9501 - binary_accuracy: 0.5312

 19/625 [..............................] - ETA: 1s - loss: 1.0678 - binary_accuracy: 0.5526  

 37/625 [>.............................] - ETA: 1s - loss: 1.0805 - binary_accuracy: 0.5524

 55/625 [=>............................] - ETA: 1s - loss: 1.0506 - binary_accuracy: 0.5580

...

674/742 [==========================>...] - ETA: 0s - loss: 0.1371 - binary_accuracy: 0.9455

693/742 [===========================>..] - ETA: 0s - loss: 0.1369 - binary_accuracy: 0.9458

712/742 [===========================>..] - ETA: 0s - loss: 0.1370 - binary_accuracy: 0.9456

731/742 [============================>.] - ETA: 0s - loss: 0.1366 - binary_accuracy: 0.9458

742/742 [==============================] - 2s 3ms/step - loss: 0.1366 - binary_accuracy: 0.9460


## **10. Evaluate the robust model's performance**

In [23]:
from sklearn.metrics import accuracy_score

pred_labels = lnl.predict(test_texts)
accuracy_score(test_labels, pred_labels)

  1/782 [..............................] - ETA: 21s

 56/782 [=>............................] - ETA: 0s 

110/782 [===>..........................] - ETA: 0s

171/782 [=====>........................] - ETA: 0s

230/782 [=======>......................] - ETA: 0s

...

636/782 [=======================>......] - ETA: 0s

697/782 [=========================>....] - ETA: 0s

755/782 [===========================>..] - ETA: 0s

782/782 [==============================] - 1s 869us/step


0.8414

## **What's next?**

Congratulation on completing this tutorial! Check out our following tutorial on using Cleanlab for tabular data classification!